In [3]:
import ast
import copy
import multiprocessing
import re
from concurrent.futures import ProcessPoolExecutor
from functools import partial

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from aocd import get_data, submit

multiprocessing.set_start_method("fork")

DAY = 19
YEAR = 2022

In [4]:
# use test data
raw_test = """Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian."""

# use real data
raw = get_data(day=DAY, year=YEAR)

print(raw_test)

Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian.


In [5]:
def parse_data(data):
    data = data.split("\n")
    blueprints = {}
    for d in data:
        bid, ore_o, cly_ore, obs_ore, obs_cly, gde_ore, gde_obs = map(int, re.findall("\d+", d))

        # bot code: cost (ore, cly, obs, gde)
        blueprints[bid] = {
            (1, 0, 0, 0): (ore_o, 0, 0, 0),  # build ore bot
            (0, 1, 0, 0): (cly_ore, 0, 0, 0),  # build cly bot
            (0, 0, 1, 0): (obs_ore, obs_cly, 0, 0),  # build obs bot
            (0, 0, 0, 1): (gde_ore, 0, gde_obs, 0),  # build gde bot
            (0, 0, 0, 0): (0, 0, 0, 0),  # collect, no bot-building
        }

    return blueprints


dummy = parse_data(raw_test)
real = parse_data(raw)

# dummy
real[17]

{(1, 0, 0, 0): (3, 0, 0, 0),
 (0, 1, 0, 0): (3, 0, 0, 0),
 (0, 0, 1, 0): (3, 9, 0, 0),
 (0, 0, 0, 1): (3, 0, 7, 0),
 (0, 0, 0, 0): (0, 0, 0, 0)}

# Part 1

In [10]:
def is_enough(mats, cost):
    return all([m >= c for m, c in zip(mats, cost)])


def potential_actions(mats, bots, blueprint, bot_limits):
    possible_actions = [action for action, cost in blueprint.items() if is_enough(mats, cost)]

    actions = []
    for a in possible_actions:  # only produce bots untill you have enough
        c1 = a == (1, 0, 0, 0) and bots[0] >= bot_limits[0]
        c2 = a == (0, 1, 0, 0) and bots[1] >= bot_limits[1]
        c3 = a == (0, 0, 1, 0) and bots[2] >= bot_limits[2]

        if not (c1 or c2 or c3):
            actions.append(a)

    return actions


def perform_action(mats, bots, blueprint, action):
    # collection
    mats = tuple([m + b for m, b in zip(mats, bots)])  # bots collect material

    # production
    mats = tuple([m - c for m, c in zip(mats, blueprint[action])])  # bot production costs material
    bots = tuple([b + a for b, a in zip(bots, action)])  # bot production increases bot amount

    return mats, bots


def dfs(blueprint, mats=None, bots=None, time_limit=24, bot_limits=None, result=None):
    if mats is None:
        mats, bots = (0, 0, 0, 0), (1, 0, 0, 0)
        bot_limits = np.array(list(blueprint.values())).max(axis=0)
        result = mats, bots

    if time_limit == 0:
        if result is None or mats[-1] > result[0][-1]:
            # print(result)
            result = (mats, bots)

        return result

    # stop early
    if mats[-1] + bots[-1] * time_limit + (time_limit * (time_limit - 1)) // 2 <= result[0][-1]:
        return result

    new_actions = potential_actions(mats, bots, blueprint, bot_limits)
    collect = (0, 0, 0, 0)

    if time_limit == 3:
        eligible_actions = set(new_actions) - {(0, 1, 0, 0)}
    elif time_limit == 2:
        eligible_actions = set(new_actions) & {(0, 0, 0, 0), (0, 0, 0, 1)}
    elif time_limit == 1:
        eligible_actions = {(0, 0, 0, 0)}
    else:
        eligible_actions = set(new_actions)

    # production line
    for a in eligible_actions - {collect}:
        new_mats, new_bots = perform_action(mats, bots, blueprint, a)
        new_time_limit = time_limit - 1
        result = dfs(blueprint, new_mats, new_bots, new_time_limit, bot_limits, result)

    # no production line
    bot_actions_to_ignore = eligible_actions - {collect}
    new_mats, new_bots, new_time_limit = mats, bots, time_limit
    while True:
        new_mats, new_bots = perform_action(new_mats, new_bots, blueprint, collect)
        new_time_limit -= 1

        updated_bot_actions = set(potential_actions(new_mats, new_bots, blueprint, bot_limits)) - eligible_actions
        if len(updated_bot_actions) > 0 or new_time_limit == 0:
            break

    if new_time_limit == 0:
        return dfs(blueprint, new_mats, new_bots, new_time_limit, bot_limits, result)

    prev_mats, prev_bots, prev_time_limit = new_mats, new_bots, new_time_limit
    for action in updated_bot_actions | {collect}:
        new_mats, new_bots = perform_action(prev_mats, prev_bots, blueprint, action)
        result = dfs(blueprint, new_mats, new_bots, prev_time_limit - 1, bot_limits, result)

    return result

In [11]:
data = real.copy()

data = np.array([[bid, bp] for bid, bp in data.items()])
bids = data[:, 0].tolist()
bps = data[:, 1].tolist()

with ProcessPoolExecutor() as executor:
    results = list(executor.map(dfs, bps))

result = sum([bid * res[0][-1] for bid, res in zip(bids, results)])
result

2301

In [28]:
# submit(result, part="a", day=DAY, year=YEAR)

# Part 2

In [25]:
data = real.copy()

blueprints = [bp for bp in list(data.values())[:3]]
dfs_v2 = partial(dfs, time_limit=32)
with ProcessPoolExecutor() as executor:
    results = list(executor.map(dfs_v2, blueprints))

result = np.prod([res[0][-1] for res in results])
result

10336

In [27]:
# submit(result, part="b", day=DAY, year=YEAR)